# Scikit-Learn IRIS Model using jsonData

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on a kubernetes cluster
 
## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [ ]:
%%writefile train_iris.py
import joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import datasets


OUTPUT_FILE = "IrisClassifier.sav"


def main():
    clf = LogisticRegression(solver="liblinear", multi_class="ovr")
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    print(f"Saving model in {OUTPUT_FILE}")
    joblib.dump(p, OUTPUT_FILE)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")

    main()

In [ ]:
!python train_iris.py

## Wrap your model with a Python wrapper

In [ ]:
%%writefile IrisClassifier.py
import joblib
import sys


def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)


class IrisClassifier(object):
    def __init__(self):
        self.model = joblib.load("IrisClassifier.sav")

    def predict(self, X, features_names):
        eprint("--------------------")
        eprint("Input dict")
        eprint(X)
        eprint("--------------------")
        ndarray = X["some_data"]["some_ndarray"]
        return self.model.predict_proba(ndarray)

## REST test

Wrap your Python model using s2i

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:1.3.0-dev_20201015134839 seldonio/sklearn-iris-jsondata:0.1

Serve model

In [ ]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 seldonio/sklearn-iris-jsondata:0.1

In [ ]:
!curl -sg http://localhost:5000/predict --data-urlencode 'json={"jsonData": {"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}}' 

Stop serving model

In [ ]:
!docker rm iris_predictor --force

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Create a seldon config file to deploy the containerized image you just created

In [ ]:
%%writefile sklearn_iris_jsondata_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris-jsondata:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

In [ ]:
!kubectl create -f sklearn_iris_jsondata_deployment.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

### Test by sending prediction calls

jsonData sent as json

In [ ]:
import json
res=!curl -s -H "Content-Type: application/json" \
    -d '{"jsonData": {"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}}' \
    "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions"
j=json.loads(res[0])
print(j)
assert(j["data"]["tensor"]["values"][1]>0.0)

jsonData sent as form-data

In [ ]:
res=!curl -s -H 'Content-Type:multipart/form-data' \
    -F jsonData='{"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}' \
    "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions"
j=json.loads(res[0])
print(j)
assert(j["data"]["tensor"]["values"][1]>0.0)

In [ ]:
!kubectl delete -f sklearn_iris_jsondata_deployment.yaml